In [1]:
def loadDataSet():
    return [[1,3,4],[2,3,5],[1,2,3,5],[2,5]]

In [2]:
def createC1(dataSet):
    C1 = []
    for transaction in dataSet:
        for item in transaction:
            if not [item] in C1:
                C1.append([item])
    C1.sort()
    return list(map(frozenset, C1))

In [3]:
def scanD(D, Ck, minSupport):
    ssCnt = {}
    for tid in D:
        for can in Ck:
            if can.issubset(tid):
                if can not in ssCnt:  
                    ssCnt[can] = 1
                else:
                    ssCnt[can] += 1  
    numItems = float(len(D))
    retList = []
    supportData = {}
    for key in ssCnt:
        support = ssCnt[key] / numItems
        if support >= minSupport:
            retList.insert(0, key)
            supportData[key] = support
    return retList, supportData


In [4]:
dataSet = loadDataSet()

In [5]:
dataSet

[[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]

In [6]:
C1 = createC1(dataSet)
C1

[frozenset({1}),
 frozenset({2}),
 frozenset({3}),
 frozenset({4}),
 frozenset({5})]

In [7]:
D = list(map(set, dataSet))
D

[{1, 3, 4}, {2, 3, 5}, {1, 2, 3, 5}, {2, 5}]

In [8]:
L1, supportData0 = scanD(D, C1, 0.5)

In [9]:
L1

[frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})]

In [10]:
def aprioriGen(Lk, k):
    retList = []
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i+1, lenLk):
            L1 = list(Lk[i])[:k-2]
            L2 = list(Lk[j])[:k-2]
            L1.sort()
            L2.sort()
            if L1 == L2:
                retList.append(Lk[i]|Lk[j])
    return retList


In [11]:
def apriori(dataSet, minSupport=0.5):
    C1 = createC1(dataSet)
    D = list(map(set, dataSet))
    L1, supportData = scanD(D, C1, minSupport)
    L = [L1]
    k = 2
    while (len(L[k-2])>0):
        Ck = aprioriGen(L[k-2], k)
        Lk, subK = scanD(D, Ck, minSupport)
        supportData.update(subK)
        L.append(Lk)
        k+=1
    return L, supportData

In [12]:
L, subData = apriori(dataSet)
L

[[frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})],
 [frozenset({2, 3}), frozenset({3, 5}), frozenset({2, 5}), frozenset({1, 3})],
 [frozenset({2, 3, 5})],
 []]

In [13]:
L[0]

[frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})]

In [14]:
L[1]

[frozenset({2, 3}), frozenset({3, 5}), frozenset({2, 5}), frozenset({1, 3})]

In [15]:
L[2]

[frozenset({2, 3, 5})]

In [16]:
L[3]

[]

In [17]:
aprioriGen(L[0], 2)

[frozenset({2, 5}),
 frozenset({3, 5}),
 frozenset({1, 5}),
 frozenset({2, 3}),
 frozenset({1, 2}),
 frozenset({1, 3})]

In [18]:
L, subData = apriori(dataSet, minSupport=0.7)
L

[[frozenset({5}), frozenset({2}), frozenset({3})], [frozenset({2, 5})], []]

In [19]:
def calcConf(freqSet, H, supportData, brl, minConf=0.7):
    prunedH = []
    for conseq in H:
        conf = supportData[freqSet]/supportData[freqSet-conseq]
        if conf >= minConf:
            print(f'{freqSet-conseq} -----> {conseq} conf:{conf}')
            brl.append((freqSet-conseq, conseq, conf))
            prunedH.append(conseq)
    return prunedH

In [ ]:
def rulesFromConseq(freqSet, H, supportData, brl, minConf=0.7):
    m = len(H[0])
    if (len(freqSet)>(m+1)):
        Hmp1 = aprioriGen(H, m+1)
        Hmp1 = calcConf(freqSet, Hmp1, supportData, brl, minConf)
        if (len(Hmp1) > 1):
            rulesFromConseq(freqSet, Hmp1, supportData, brl, minConf)


In [23]:
# def generateRules(L, supportData, minConf=0.7):
#     bigRuleList = []
#     for i in range(1, len(L)):
#         for freqSet in L[i]:
#             H1 = [frozenset([item]) for item in freqSet]
#             if i > 1:
#                 rulesFromConseq(freqSet, H1, supportData, bigRuleList, minConf)
#     return bigRuleList

def generateRules(L, supportData, minConf=0.7):
    bigRuleList = []
    for i in range(1, len(L)):
        for freqSet in L[i]:
            H1 = [frozenset([item]) for item in freqSet]
            if i > 1:
                rulesFromConseq(freqSet, H1, supportData, bigRuleList, minConf)
            else:
                calcConf(freqSet, H1, supportData, bigRuleList, minConf)
    return bigRuleList


In [24]:
L, subData = apriori(dataSet, minSupport=0.5)
rules = generateRules(L, subData, minConf=0.7)
rules

frozenset({5}) -----> frozenset({2}) conf:1.0
frozenset({2}) -----> frozenset({5}) conf:1.0
frozenset({1}) -----> frozenset({3}) conf:1.0


[(frozenset({5}), frozenset({2}), 1.0),
 (frozenset({2}), frozenset({5}), 1.0),
 (frozenset({1}), frozenset({3}), 1.0)]

In [25]:
rules = generateRules(L, subData, minConf=0.5)

frozenset({3}) -----> frozenset({2}) conf:0.6666666666666666
frozenset({2}) -----> frozenset({3}) conf:0.6666666666666666
frozenset({5}) -----> frozenset({3}) conf:0.6666666666666666
frozenset({3}) -----> frozenset({5}) conf:0.6666666666666666
frozenset({5}) -----> frozenset({2}) conf:1.0
frozenset({2}) -----> frozenset({5}) conf:1.0
frozenset({3}) -----> frozenset({1}) conf:0.6666666666666666
frozenset({1}) -----> frozenset({3}) conf:1.0
frozenset({5}) -----> frozenset({2, 3}) conf:0.6666666666666666
frozenset({3}) -----> frozenset({2, 5}) conf:0.6666666666666666
frozenset({2}) -----> frozenset({3, 5}) conf:0.6666666666666666


In [27]:
mushDataSet = [line.split() for line in open('mushroom.dat').readlines()]

In [28]:
L, subData = apriori(mushDataSet, minSupport=0.3)

In [29]:
for item in L[1]:
    if item.intersection('2'):
        print(item)

frozenset({'28', '2'})
frozenset({'2', '53'})
frozenset({'23', '2'})
frozenset({'34', '2'})
frozenset({'36', '2'})
frozenset({'59', '2'})
frozenset({'2', '63'})
frozenset({'67', '2'})
frozenset({'76', '2'})
frozenset({'85', '2'})
frozenset({'86', '2'})
frozenset({'90', '2'})
frozenset({'2', '93'})
frozenset({'2', '39'})


In [31]:
for item in L[3]:
    if item.intersection('1'):
        print(item)

frozenset({'1', '36', '24', '110'})
frozenset({'1', '34', '24', '110'})
frozenset({'1', '85', '24', '110'})
frozenset({'1', '86', '24', '110'})
frozenset({'1', '24', '90', '110'})
frozenset({'1', '34', '36', '24'})
frozenset({'1', '85', '34', '24'})
frozenset({'1', '85', '36', '24'})
frozenset({'1', '86', '34', '24'})
frozenset({'1', '86', '36', '24'})
frozenset({'1', '86', '85', '24'})
frozenset({'1', '34', '24', '90'})
frozenset({'1', '36', '24', '90'})
frozenset({'1', '85', '24', '90'})
frozenset({'1', '86', '24', '90'})
frozenset({'1', '36', '34', '110'})
frozenset({'1', '85', '34', '110'})
frozenset({'1', '86', '34', '110'})
frozenset({'1', '34', '90', '110'})
frozenset({'1', '85', '36', '110'})
frozenset({'1', '85', '86', '110'})
frozenset({'1', '85', '90', '110'})
frozenset({'1', '86', '36', '110'})
frozenset({'1', '86', '90', '110'})
frozenset({'1', '36', '90', '110'})
frozenset({'1', '85', '36', '34'})
frozenset({'1', '86', '36', '34'})
frozenset({'1', '86', '85', '34'})
froze